In [2]:
%reload_ext autoreload
%autoreload 3

import sys
import warnings
import re
import os
import glob
import pandas
import time
import pathlib
from shutil import rmtree
warnings.filterwarnings('ignore')

#sys.path.append("G:\Outros computadores\Meu computador\github\circadipy\src")
sys.path.append("E:\github\circadipy\src")
import chrono_reader as chr
import chrono_plotter as chp
import chrono_rithm as chrt

PyMICE library v. 1.2.1

The library is available under GPL3 license; we ask that reference to our paper
as well as to the library itself is provided in any published research making
use of PyMICE. Please run:

>>> print(pm.__REFERENCING__)

for more information (given that the library is imported as `pm`).




In [9]:
experiment_number = 5
#root_folder = "G:\\Outros computadores\\Meu computador\\university\\doctorade\\future_care\\chronobiology\\protocols\\data_expto_" + str(experiment_number)
root_folder = 'E:\\university\\doctorade\\future_care\\chronobiology\\protocols\\data_expto_' + str(experiment_number)
data_folder = root_folder + "\\data"

results_folder = root_folder + "\\results_without_norm"

print(data_folder)

E:\university\doctorade\future_care\chronobiology\protocols\data_expto_5\data


In [10]:
# Build a dictionary with the paths to the data files to read 1 animal as example
if experiment_number == 1:
    labels = ['1_control_dl', '2_control_dd', '3_control_dd', '4_control_dd', '5_hypo_dd', '6_hypo_dd', '7_hypo_dl']
    name_labels = ['1_control_dl', '2_control_dd', '2_control_dd', '2_control_dd', '3_hypo_dd', '3_hypo_dd', '4_hypo_dl'] 
    cycles = ['DL', 'DD', 'DD', 'DD', 'DD', 'DD', 'DL']                                                              
elif experiment_number == 5:
    labels = ['1_control_dl', '2_control_dd', '3_control_dd', '4_hypo_induction', '5_hypo_dd', '6_hypo_dl']
    name_labels = ['1_control_dl', '2_control_dd', '2_control_dd', '3_hypo_dd', '3_hypo_dd', '4_hypo_dl']
    cycles = ['DL', 'DD', 'DD', 'DD', 'DD', 'DL']

protocol_files = {}
for label in labels:
    protocol_files[label] = {'folder': data_folder + "\\" + label}

for key in protocol_files.keys():
    file_in_dir = os.listdir(protocol_files[key]['folder'])
    protocol_files[key]['files'] = [file for file in file_in_dir if file.endswith(".asc")]
    protocol_files[key]['files'].sort()

animals = {}

if experiment_number == 1:
    number_of_protocols = 8
elif experiment_number == 5: 
    number_of_protocols = 8

for count in range(0, number_of_protocols):
    if count < 10 - 1:
        animals["animal_0" + str(count + 1)] = {}
        animals["animal_0" + str(count + 1)]['act_files'] = []
        animals["animal_0" + str(count + 1)]['tmp_files'] = []
    else:
        animals["animal_" + str(count + 1)] = {}
        animals["animal_" + str(count + 1)]['act_files'] = []
        animals["animal_" + str(count + 1)]['tmp_files'] = []

for key in protocol_files.keys():
    for count in range(0, number_of_protocols):
        if count < 10 - 1:
            animals["animal_0" + str(count + 1)]['act_files'].append(protocol_files[key]['folder'] + "\\" + protocol_files[key]['files'][count])
            #animals["animal_0" + str(count + 1)]['results_folder'] = results_folder + "\\" + "animal_0" + str(count + 1)
        else:
            animals["animal_" + str(count + 1)]['act_files'].append(protocol_files[key]['folder'] + "\\" + protocol_files[key]['files'][count])
            #animals["animal_" + str(count + 1)]['results_folder'] = results_folder + "\\" + "animal_" + str(count + 1)
    for count in range(number_of_protocols, number_of_protocols*2):
        if count < 10 + number_of_protocols - 1:
            animals["animal_0" + str(count - number_of_protocols + 1)]['tmp_files'].append(protocol_files[key]['folder'] + "\\" + protocol_files[key]['files'][count])
        else:
            animals["animal_" + str(count - number_of_protocols + 1)]['tmp_files'].append(protocol_files[key]['folder'] + "\\" + protocol_files[key]['files'][count])

display(animals)

{'animal_01': {'act_files': ['E:\\university\\doctorade\\future_care\\chronobiology\\protocols\\data_expto_5\\data\\1_control_dl\\control_dl_ale_animal_01.asc',
   'E:\\university\\doctorade\\future_care\\chronobiology\\protocols\\data_expto_5\\data\\2_control_dd\\control_dd_part1_ale_animal_01.asc',
   'E:\\university\\doctorade\\future_care\\chronobiology\\protocols\\data_expto_5\\data\\3_control_dd\\control_dd_part2_ale_animal_01.asc',
   'E:\\university\\doctorade\\future_care\\chronobiology\\protocols\\data_expto_5\\data\\4_hypo_induction\\hypo_induction_ale_animal_01.asc',
   'E:\\university\\doctorade\\future_care\\chronobiology\\protocols\\data_expto_5\\data\\5_hypo_dd\\hypo_dd_ale_animal_01.asc',
   'E:\\university\\doctorade\\future_care\\chronobiology\\protocols\\data_expto_5\\data\\6_hypo_dl\\hypo_dl_ale_animal_01.asc'],
  'tmp_files': ['E:\\university\\doctorade\\future_care\\chronobiology\\protocols\\data_expto_5\\data\\1_control_dl\\control_dl_temp_animal_01.asc',
   'E:

In [17]:
# SET THE PARAMETERS TO ANALYZE --------

filter = True
filter_type = 'moving_average'
normalize = False
normalize_per_day = False
animal_select = list(range(1, len(animals) + 1))
#animal_select = []
zt_0_time = 20  

# SET THE PARAMETERS TO ANALYZE --------

protocols = []
animals_keys = []
num_animals = len(list(animals.keys()))

select_index = [s - 1 for s in animal_select]

In [18]:
os.listdir(root_folder)

['data',
 'desktop.ini',
 'results_filtered_w3_normalized',
 'results_without_norm']

In [19]:
def reset_folder(folder):
    for root, dirs, files in os.walk(folder):
        for f in files:
            os.unlink(os.path.join(root, f))
        for d in dirs:
            rmtree(os.path.join(root, d))

if results_folder.split("\\")[-1] not in os.listdir(root_folder):
    os.mkdir(results_folder)
    print("Folder results created!")

for count, key in enumerate(list(animals.keys())):
    if count in select_index:
        if key not in os.listdir(results_folder):
            os.mkdir(results_folder + '\\' + key)
            print("Folder " + key + " created!")
        else:
            #reset_folder(results_folder + '\\' + key)
            print("Folder " + key + " cleaned!")

Folder animal_01 cleaned!
Folder animal_02 created!
Folder animal_03 created!
Folder animal_04 created!
Folder animal_05 created!
Folder animal_06 cleaned!
Folder animal_07 created!
Folder animal_08 created!


In [20]:
pandas.options.display.max_rows = 10000

if experiment_number == 1:
    labels_dict = [{'cycle_types': ['DL'], 'test_labels': ['1_control_dl'], 'cycle_days': []},
                   {'cycle_types': ['DD'], 'test_labels': ['2_control_dd'], 'cycle_days': []},
                   {'cycle_types': ['DD'], 'test_labels': ['2_control_dd'], 'cycle_days': []},
                   {'cycle_types': ['DD'], 'test_labels': ['2_control_dd'], 'cycle_days': []},
                   {'cycle_types': ['DD'], 'test_labels': ['3_induction_dd'], 'cycle_days': []},
                   {'cycle_types': ['DD'], 'test_labels': ['4_hypo_dd'], 'cycle_days': []},
                   {'cycle_types': ['DL'], 'test_labels': ['5_hypo_dl'], 'cycle_days': []}]                                                              
elif experiment_number == 5:
    labels_dict = [{'cycle_types': ['DL'], 'test_labels': ['1_control_dl'], 'cycle_days': []},
                   {'cycle_types': ['DD'], 'test_labels': ['2_control_dd'], 'cycle_days': []},
                   {'cycle_types': ['DD'], 'test_labels': ['2_control_dd'], 'cycle_days': []},
                   {'cycle_types': ['DD'], 'test_labels': ['3_induction_dd'], 'cycle_days': []},
                   {'cycle_types': ['DD'], 'test_labels': ['4_hypo_dd'], 'cycle_days': []},
                   {'cycle_types': ['DL'], 'test_labels': ['5_hypo_dl'], 'cycle_days': []}]               

description = 'Running analysis with the following parameters:\n - Filter: ' + str(filter) + '\n - Filter type: ' + filter_type + '\n - Normalize: ' + str(normalize) + '\n - Normalize per day: ' + str(normalize_per_day) + '\n - Animal select: ' + str(animal_select) + '\n - ZT 0 time: ' + str(zt_0_time) + '\n - Labels dict: ' + str(labels_dict)
print(description)
print('\n')

for count, animal in enumerate(animals):
    if count in select_index:
        for label in range(0, len(labels_dict)):
            if label == 0:
                animals[animal]['tmp_protocol'] = chr.read_protocol(animal, animals[animal]['tmp_files'][label], zt_0_time = zt_0_time, labels_dict = labels_dict[label], type = 'endocrino', consider_first_day = False)               
                nans_to_correct_act = animals[animal]['tmp_protocol'].get_is_nan_indexes()
                animals[animal]['tmp_protocol'].resample('1H', method = 'last')

                animals[animal]['act_protocol'] = chr.read_protocol(animal, animals[animal]['act_files'][label], zt_0_time = zt_0_time, labels_dict = labels_dict[label], type = 'endocrino', consider_first_day = False, set_nans = nans_to_correct_act)               
                animals[animal]['act_protocol'].resample('1H', method = 'sum')
            else:
                next_protocol = chr.read_protocol(animal, animals[animal]['tmp_files'][label], zt_0_time = zt_0_time, labels_dict = labels_dict[label], type = 'endocrino', consider_first_day = True)
                nans_to_correct_act = next_protocol.get_is_nan_indexes()
                next_protocol.resample('1H', method = 'last')
                animals[animal]['tmp_protocol'].concat_protocols(next_protocol, method = 'last')

                next_protocol = chr.read_protocol(animal, animals[animal]['act_files'][label], zt_0_time = zt_0_time, labels_dict = labels_dict[label], type = 'endocrino', consider_first_day = True, set_nans = nans_to_correct_act)
                next_protocol.resample('1H', method = 'sum')
                animals[animal]['act_protocol'].concat_protocols(next_protocol, method = 'sum')

        if filter:
            animals[animal]['tmp_protocol'].apply_filter(window = 3, type = filter_type, order = 2, reverse = False)
            animals[animal]['act_protocol'].apply_filter(window = 3, type = filter_type, order = 2, reverse = False)
        if normalize:
            animals[animal]['tmp_protocol'].normalize_data(type = 'minmax', per_day = normalize_per_day)
            animals[animal]['act_protocol'].normalize_data(type = 'minmax', per_day = normalize_per_day)
        
        animals[animal]['tmp_protocol'].correct_labels()
        animals[animal]['act_protocol'].correct_labels()
        start_date_tmp = animals[animal]['tmp_protocol'].start_date
        end_date_tmp = animals[animal]['tmp_protocol'].end_date
        start_date_act = animals[animal]['act_protocol'].start_date
        end_date_act = animals[animal]['act_protocol'].end_date

        if start_date_tmp == start_date_act and end_date_tmp == end_date_act:
            print('Animal: ' + animal + ', start date: ' + str(start_date_act) + ', end date: ' + str(end_date_act))
        else:
            print('SOMETHING IS WRONG WITH THE PROTOCOLS OF ANIMAL ' + animal + '!!!')



Running analysis with the following parameters:
 - Filter: True
 - Filter type: moving_average
 - Normalize: False
 - Normalize per day: False
 - Animal select: [1, 2, 3, 4, 5, 6, 7, 8]
 - ZT 0 time: 20
 - Labels dict: [{'cycle_types': ['DL'], 'test_labels': ['1_control_dl'], 'cycle_days': []}, {'cycle_types': ['DD'], 'test_labels': ['2_control_dd'], 'cycle_days': []}, {'cycle_types': ['DD'], 'test_labels': ['2_control_dd'], 'cycle_days': []}, {'cycle_types': ['DD'], 'test_labels': ['3_induction_dd'], 'cycle_days': []}, {'cycle_types': ['DD'], 'test_labels': ['4_hypo_dd'], 'cycle_days': []}, {'cycle_types': ['DL'], 'test_labels': ['5_hypo_dl'], 'cycle_days': []}]


Animal: animal_01, start date: 2022-06-08 00:00:00, end date: 2022-09-08 15:00:00
Animal: animal_02, start date: 2022-06-08 00:00:00, end date: 2022-09-08 15:00:00
Animal: animal_03, start date: 2022-06-08 00:00:00, end date: 2022-09-08 15:00:00
Animal: animal_04, start date: 2022-06-08 00:00:00, end date: 2022-09-08 15:00:0

In [21]:
for count, animal in enumerate(animals):
    if count in select_index:
        save_folder = results_folder + '\\' + animal
        chp.actogram_bar(animals[animal]['tmp_protocol'], first_hour = 18, save_folder = save_folder, save_suffix = 'tmp', adjust_figure = [1.2, 0.85, 0.85, 0.2, 0.15])
        chp.actogram_colormap(animals[animal]['tmp_protocol'], first_hour = 18, save_folder = save_folder, save_suffix = 'tmp', adjust_figure = [1.2, 0.85, 0.85, 0.2, 0.15])
        #chp.data_periodogram(animals[animal]['tmp_protocol'], time_shape = 'continuous', method = 'periodogram', max_period = 48, unit_of_measurement = '°C', save_folder = save_folder, save_suffix = 'tmp_periodogram')
        #chp.data_periodogram(animals[animal]['tmp_protocol'], time_shape = 'continuous', method = 'welch', max_period = 48, unit_of_measurement = '°C', save_folder = save_folder, save_suffix = 'tmp_welch')
        print(animal.replace('_', ' ').capitalize() + ' temperature actograms saved!')

        # chp.actogram_bar(animals[animal]['act_protocol'], first_hour = 18, save_folder = save_folder, save_suffix = 'act')
        # chp.actogram_colormap(animals[animal]['act_protocol'], first_hour = 18, save_folder = save_folder, save_suffix = 'act')
        # chp.data_periodogram(animals[animal]['act_protocol'], time_shape = 'continuous', method = 'periodogram', max_period = 48, unit_of_measurement = 'Counts', save_folder = save_folder, save_suffix = 'act_periodogram')
        # chp.data_periodogram(animals[animal]['act_protocol'], time_shape = 'continuous', method = 'welch', max_period = 48, unit_of_measurement = 'Counts', save_folder = save_folder, save_suffix = 'act_welch')
        # print(animal.replace('_', ' ').capitalize() + ' activity actograms saved!')

Animal 01 temperature actograms saved!
Animal 02 temperature actograms saved!
Animal 03 temperature actograms saved!
Animal 04 temperature actograms saved!
Animal 05 temperature actograms saved!
Animal 06 temperature actograms saved!
Animal 07 temperature actograms saved!
Animal 08 temperature actograms saved!


In [15]:
dict = {'time_shape': 'continuous', 
        'step': 0.01, 
        'start_time': 22, 
        'end_time': 26, 
        'n_components': [1]}

best_models = []

for count, animal in enumerate(animals):
    if count in select_index:
        init = time.time()

        save_folder = results_folder + '\\' + animal

        tmp_best_models = chrt.fit_cosinor(animals[animal]['tmp_protocol'], dict = dict, save_folder = save_folder, save_suffix = 'tmp')
        tmp_best_models_fixed = chrt.fit_cosinor_fixed_period(animals[animal]['tmp_protocol'], tmp_best_models, save_folder = save_folder, save_suffix = 'tmp')    
        animals[animal]['tmp_best_models'] = tmp_best_models
        animals[animal]['tmp_best_models_fixed'] = tmp_best_models_fixed    

        act_best_models = chrt.fit_cosinor(animals[animal]['act_protocol'], dict = dict, save_folder = save_folder, save_suffix = 'act')
        act_best_models_fixed = chrt.fit_cosinor_fixed_period(animals[animal]['act_protocol'], act_best_models, save_folder = save_folder, save_suffix = 'act')  
        animals[animal]['act_best_models'] = act_best_models
        animals[animal]['act_best_models_fixed'] = act_best_models_fixed    

        end = time.time() - init

        print("Cosinor fitted to " + animal.replace('_', ' ') + " and results saved!")
        print("Time elapsed: " + "{:.2f}".format(end) + " seconds")

Cosinor fitted to animal 01 and results saved!
Time elapsed: 93.04 seconds
Cosinor fitted to animal 02 and results saved!
Time elapsed: 93.82 seconds
Cosinor fitted to animal 03 and results saved!
Time elapsed: 93.06 seconds
Cosinor fitted to animal 04 and results saved!
Time elapsed: 93.91 seconds
Cosinor fitted to animal 05 and results saved!
Time elapsed: 92.84 seconds
Cosinor fitted to animal 06 and results saved!
Time elapsed: 93.71 seconds
Cosinor fitted to animal 07 and results saved!
Time elapsed: 94.86 seconds
Cosinor fitted to animal 08 and results saved!
Time elapsed: 93.20 seconds
Cosinor fitted to animal 09 and results saved!
Time elapsed: 93.18 seconds
Cosinor fitted to animal 10 and results saved!
Time elapsed: 94.38 seconds


In [16]:
for count, animal in enumerate(animals):
    if count in select_index:
        init = time.time()
        
        save_folder = results_folder + '\\' + animal

        chrt.plot_cosinor(animals[animal]['tmp_protocol'], animals[animal]['tmp_best_models_fixed'], save_folder = save_folder, save_suffix = 'tmp')
        chrt.plot_cosinor_model_overview(animals[animal]['tmp_protocol'], animals[animal]['tmp_best_models'], save_folder = save_folder, save_suffix = 'tmp')
        chrt.plot_cosinor_model_over_signal(animals[animal]['tmp_protocol'], animals[animal]['tmp_best_models'], position = 'head', mv_avg_window = 1, save_folder = save_folder, save_suffix = 'tmp')

        chrt.plot_cosinor(animals[animal]['act_protocol'], animals[animal]['act_best_models_fixed'], save_folder = save_folder, save_suffix = 'act')
        chrt.plot_cosinor_model_overview(animals[animal]['act_protocol'], animals[animal]['act_best_models'], save_folder = save_folder, save_suffix = 'act')
        chrt.plot_cosinor_model_over_signal(animals[animal]['act_protocol'], animals[animal]['act_best_models'], position = 'head', mv_avg_window = 1, save_folder = save_folder, save_suffix = 'act')


        end = time.time() - init

        print("Cosinor overview " + animal.replace('_', ' ') + " saved!")
        print("Time elapsed: " + "{:.2f}".format(end) + " seconds")

Cosinor overview animal 01 saved!
Time elapsed: 13.58 seconds
Cosinor overview animal 02 saved!
Time elapsed: 13.57 seconds
Cosinor overview animal 03 saved!
Time elapsed: 13.90 seconds
Cosinor overview animal 04 saved!
Time elapsed: 15.40 seconds
Cosinor overview animal 05 saved!
Time elapsed: 14.40 seconds
Cosinor overview animal 06 saved!
Time elapsed: 15.80 seconds
Cosinor overview animal 07 saved!
Time elapsed: 15.09 seconds
Cosinor overview animal 08 saved!
Time elapsed: 13.66 seconds
Cosinor overview animal 09 saved!
Time elapsed: 13.96 seconds
Cosinor overview animal 10 saved!
Time elapsed: 14.67 seconds


In [ ]:
# for d in [1,3,5]:
#     dict = {'day_window': d,
#             'step': 0.01,
#             'start_time': 22,
#             'end_time': 26,
#             'n_components': [1]}

#     best_models_per_day_activity = []
#     for animal in animals:
#         init = time.time()

#         save_folder = results_folder + '\\' + animal

#         best_models_per_day = chrt.fit_cosinor_per_day(animals[animal]['protocol'], dict = dict, plot = True, save_folder = save_folder)
#         chrt.plot_cosinor_per_day(animals[animal]['protocol'], best_models_per_day = best_models_per_day, day_window = dict['day_window'], save_folder = save_folder)
#         animals[animal]['best_models_per_day_activity'] = best_models_per_day

#         end = time.time() - init

#         print("Cosinor fitted to each day window (" + str(d) + 'days(s)) ' + animal.replace('_', ' ') + " and results saved!")
#         print("Time elapsed: " + "{:.2f}".format(end) + " seconds")